导入所需包

In [1]:
import numpy as np

实现Romberg求积法

注意到在表4.2中，第$k$层的$T$的计算过程至多只会调用第$k-1$层的数据（某种意义上说，相当于具有马尔可夫性）

因此，可以根据该“马尔科夫链”在一步转移过程中所需的空间大小进行设计，避免历史信息冗余造成的空间浪费

在这里，为减小空间复杂度，仅采用一个线性表存储$T$的值，当然，作为代价，时间复杂度或许会有上升，但在这里感受并不明显

现举例如下，其中“ [ ”表示线性表的表头：

初始化时

1)生成$T_{0,0}$时，线性表：[ $T_{0,0}$

$k=1$时

2)生成$T_{0,0}\to T_{1,0}$时，线性表：[ $T_{0,0}$, $T_{1,0}$

3)生成$(T_{0,0},T_{1,0})\to T_{0,1}$时，线性表：[ $T_{0,1}$, $T_{1,0}$

$k=2$时

4)生成$T_{1,0}\to T_{2,0}$时，线性表：[ $T_{0,1}$, $T_{1,0}$, $T_{2,0}$

5)生成$(T_{1,0},T_{2,0})\to T_{1,1}$时，线性表：[ $T_{0,1}$, $T_{1,1}$, $T_{2,0}$

6)生成$(T_{0,1},T_{1,1})\to T_{0,2}$时，线性表：[ $T_{0,2}$, $T_{1,1}$, $T_{2,0}$

余下类似

In [2]:
def romberg(integrand, LowerBound, UpperBound):
    f, a, b = integrand, LowerBound, UpperBound
    h = b - a
    k = 1
    T = []
    epsilon = 1e-10
    
    T.append(h / 2 * (f(a) + f(b))) # 计算 T(0, 0)
    while True:
        T_old = T[0] # 记录 T(0, k-1)
        I = np.arange(1, 2**(k-1) + 1)
        T_delta = h * f(a + (I-1/2) * h).sum()
        T.append((T[k - 1] + T_delta) / 2) # 计算 T(k, 0)
        for m in range(1, k + 1):
            T[k - m] = 4 ** m * T[k + 1 - m] - T[k - m]
            T[k - m] = T[k - m] / (4 ** m - 1) # 计算 T(k-m, m)
        if np.abs(T[0] - T_old) <= epsilon:
            break
        else:
            k = k + 1
            h = h / 2
    
    print('Romberg求积法: T(0, k) = {0}, k = {1}'.format(T[0], k))

实现复化左矩形数值积分算法

注意这里也使用了类似马尔科夫链的思想，只采用了$L_{new}$和$L_{old}$两个变量来存储近似积分数据

In [3]:
def LeftRectangle(integrand, LowerBound, UpperBound):
    f, a, b = integrand, LowerBound, UpperBound
    h = b - a
    n = 1
    epsilon = 1e-8
    
    I = np.arange(n)
    L_new = h * f(a + I * h).sum()
    while True:
        n = n + 1
        L_old = L_new
        h = (b - a) / n
        I = np.arange(n)
        L_new = h * f(a + I * h).sum()
        if np.abs(L_new - L_old) <= epsilon:
            break
    n = n - 1
    
    print('复化左矩形公式: L(n + 1) = {0}, n = {1}'.format(L_new, n))

实现复化右矩形数值积分算法

In [4]:
def RightRectangle(integrand, LowerBound, UpperBound):
    f, a, b = integrand, LowerBound, UpperBound
    h = b - a
    n = 1
    epsilon = 1e-8
    
    I = np.arange(n)
    R_new = h * f(a + (I+1) * h).sum()
    while True:
        n = n + 1
        R_old = R_new
        h = (b - a) / n
        I = np.arange(n)
        R_new = h * f(a + (I+1) * h).sum()
        if np.abs(R_new - R_old) <= epsilon:
            break
    n = n - 1
    
    print('复化右矩形公式: L(n + 1) = {0}, n = {1}'.format(R_new, n))

实现复化中矩形数值积分算法

In [5]:
def MedianRectangle(integrand, LowerBound, UpperBound):
    f, a, b = integrand, LowerBound, UpperBound
    h = b - a
    n = 1
    epsilon = 1e-8
    
    I = np.arange(n)
    M_new = h * f(a + (I+1/2) * h).sum()
    while True:
        n = n + 1
        M_old = M_new
        h = (b - a) / n
        I = np.arange(n)
        M_new = h * f(a + (I+1/2) * h).sum()
        if np.abs(M_new - M_old) <= epsilon:
            break
    n = n - 1
    
    print('复化中矩形公式: L(n + 1) = {0}, n = {1}'.format(M_new, n))

分别将题给三个被积函数及相应的积分区间代入上述算法

In [6]:
if __name__ == '__main__':
    print('x -> sin(x) / x from 1 to 2')
    f1 = lambda x: np.sin(x) / x
    romberg(f1, 1, 2)
    LeftRectangle(f1, 1, 2)
    RightRectangle(f1, 1, 2)
    MedianRectangle(f1, 1, 2)
    
    print('\nx -> 1 / log(x) ** 3 from 2 to 3')
    f2 = lambda x: 1 / np.log(x) ** 3
    romberg(f2, 2, 3)
    LeftRectangle(f2, 2, 3)
    RightRectangle(f2, 2, 3)
    MedianRectangle(f2, 2, 3)
    
    print('\nx -> cos(x) from 0 to pi / 2')
    romberg(np.cos, 0, np.pi / 2)
    LeftRectangle(np.cos, 0, np.pi / 2)
    RightRectangle(np.cos, 0, np.pi / 2)
    MedianRectangle(np.cos, 0, np.pi / 2)

x -> sin(x) / x from 1 to 2
Romberg求积法: T(0, k) = 0.6593299064355116, k = 4
复化左矩形公式: L(n + 1) = 0.659373872926283, n = 4398
复化右矩形公式: L(n + 1) = 0.6592859387886618, n = 4398
复化中矩形公式: L(n + 1) = 0.6593304137267226, n = 104

x -> 1 / log(x) ** 3 from 2 to 3
Romberg求积法: T(0, k) = 1.4751144146938298, k = 7
复化左矩形公式: L(n + 1) = 1.4752204357487875, n = 10604
复化右矩形公式: L(n + 1) = 1.4750083922543666, n = 10603
复化中矩形公式: L(n + 1) = 1.4751126069941638, n = 365

x -> cos(x) from 0 to pi / 2
Romberg求积法: T(0, k) = 1.0000000000000004, k = 5
复化左矩形公式: L(n + 1) = 1.000088612768131, n = 8862
复化右矩形公式: L(n + 1) = 0.9999113919945584, n = 8863
复化中矩形公式: L(n + 1) = 1.0000013594509356, n = 274


注意：Romberg求积法选用精度为$10^{-10}$，其余三个复化矩形公式为$10^{-8}$